In [1]:
#imports
import pandas as pd
from datetime import datetime, timedelta

#global
camposOutliers = ['Quantity','UnitPrice']
df = pd.read_csv("../data/data.csv", encoding="ISO-8859-1")
dfLenOriginal= len(df)

print("Resumen de los datos:")
df.info()

print("\nPrimeras filas:")
print(df.head(20))


Resumen de los datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

Primeras filas:
   InvoiceNo StockCode                          Description  Quantity  \
0     536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1     536365     71053                  WHITE METAL LANTERN         6   
2     536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3     536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE       

In [14]:
# tratamiento CustomerID null
print("\nAntes:")
print(df['CustomerID'].isnull().sum())

# agrupamos por InvoiceNo , y usamos ffill y bfill para rellenar, los valores nulls que no se rellenen les da un valor nuevo unico
df['CustomerID'] = df.groupby(['InvoiceNo'])['CustomerID'].ffill().bfill()

print("\nDespues:")
print(df['CustomerID'].isnull().sum())


Antes:
135080

Despues:
0


In [16]:
# tratamiento Description null
print("\nAntes:")
print(df['Description'].isnull().sum())

# agrupamos por stockcode, y usamos ffill para rellenar con el anterior valor dentro del grupo, para los primer valor en null tambien uso bfill
df['Description'] = df.groupby(['StockCode'])['Description'].transform(lambda group: group.ffill().bfill().fillna('No description')) 

print("\nDespues:")
print(df['Description'].isnull().sum())



Antes:
1454


C:\Users\adri_\AppData\Local\Temp\ipykernel_22984\2989794263.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Description'] = df.groupby(['StockCode'])['Description'].transform(lambda group: group.ffill().bfill().fillna('No description'))



Despues:
0


In [17]:
# tratamiento duplicados
print("\nNumero de duplicados antes:")
print(df.duplicated().sum())

df = df.drop_duplicates()

print("\nNumero de duplicados despues:")
print(df.duplicated().sum())


# tratamiento de negativos, los guardo en otro df, y me los quito del principal
mask = df[camposOutliers] < 0
dfNegativos = df[mask.any(axis=1)]
df= df[~mask.any(axis=1)]

print("\nNumero de negativos:")
print(mask.sum())
print("\nDatos negativos:")
print(dfNegativos.info())


Numero de duplicados antes:
5268

Numero de duplicados despues:
0

Numero de negativos:
Quantity     10587
UnitPrice        2
dtype: int64


In [20]:
# tratamiento fechas invalidas por formato
def fecha_valida(fecha, formato="%m/%d/%Y %H:%M"):
    try:
        datetime.strptime(fecha, formato)  # Intentamos convertir la fecha
        return True
    except ValueError:
        return False


fechasValidas= df['InvoiceDate'].apply(fecha_valida)
fechasInvalidas = df[~fechasValidas]

print("\nFechas invalidas:")
print(len(fechasInvalidas))

# Borramos las invalidas
df = df[fechasValidas]



Fechas invalidas:
0


In [186]:
# tratamiento fechas invalidas por fecha
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format="%m/%d/%Y %H:%M")

# pd.to_datetime usa el formato %Y/%m/%d, el output se pude cambiar con strftime
fechaInicio =  pd.to_datetime('2010/12/1 00:01')
fechaFin =  pd.to_datetime('2011/12/9 23:59')

fechasBoolean = (df['InvoiceDate'] < fechaInicio) | (df['InvoiceDate'] > fechaFin)
fechasFueraLimite=df[fechasBoolean]
print("\nFechas fuera de limite:")
print(len(fechasFueraLimite))

# quitamos del DF
df = df[~fechasBoolean]


Fechas fuera de limite:
0


In [146]:
# para los outliers, usamos el rango intercuartil (IQR)
q1 = df[camposOutliers].quantile(0.25)
q3 = df[camposOutliers].quantile(0.75)
iqr = q3- q1

#ajuste de rango para 90%//95%
rango=2.5
limiteInferior = q1 - (rango * iqr)
limiteSuperior = q3 + (rango * iqr)

outliersBoolean = (df[camposOutliers] >= limiteInferior) & (df[camposOutliers] <= limiteSuperior)
dfFiltrado = df[outliersBoolean.all(axis=1)]

# porcentaje filtarado
print(f"Porcentaje de datos retenidos: {len(dfFiltrado) / len(df) * 100:.2f}%")

Porcentaje de datos retenidos: 91.36%


In [ ]:
# Añado columna de total
df['Total'] = df[Quantity] * df[UnitPrice]


In [ ]:
# Total datos limpiados
print(f"/n Antes de limpieza: {dfLenOriginal}")
print(f"/n Despues de limpieza: {len(df)}")
print(f"/n Total de columnas eliminadas: {dfLenOriginal-len(df)}")

print("Resumen de los datos:")
df.info()


# Guardar datos limpios en un nuevo archivo CSV
df.to_csv('../data/data_limpia.csv', index=False)
